In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.utils import np_utils, to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.datasets import reuters

Using TensorFlow backend.


# Load & prep data

In [25]:
(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = reuters.load_data(num_words=1000, test_split=0.2)

In [26]:
print(X_train_raw[0])

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [27]:
print(y_train_raw[0])

3


In [28]:
num_classes = np.max(y_train_raw) + 1
print("Categories: {}".format(num_classes))

Categories: 46


In [29]:
tokenizer = Tokenizer(num_words=1000)
X_train = tokenizer.sequences_to_matrix(X_train_raw, mode='binary')
X_test = tokenizer.sequences_to_matrix(X_test_raw, mode='binary')

In [30]:
y_train = to_categorical(y_train_raw, num_classes)
y_test = to_categorical(y_test_raw, num_classes)

# Simple model

## Build model

Pretty straight forward 3 dense layers + dropout.

In [51]:
mdl = Sequential()

mdl.add(Dense(256, input_shape=(1000,)))
mdl.add(Dense(256))
mdl.add(Dense(256))
mdl.add(Activation('relu'))
mdl.add(Dropout(0.5))

mdl.add(Dense(num_classes))
mdl.add(Activation('softmax'))

In [52]:
mdl.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

## Train model

In [53]:
mdl.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/1
8982/8982 [==============================] - 2s 188us/step - loss: 0.0427 - acc: 0.9889 - val_loss: 0.0359 - val_acc: 0.9903


In [54]:
mdl.fit(X_train, y_train, batch_size=64, epochs=12, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/12
8982/8982 [==============================] - 1s 106us/step - loss: 0.0294 - acc: 0.9919 - val_loss: 0.0334 - val_acc: 0.9908
Epoch 2/12
8982/8982 [==============================] - 1s 107us/step - loss: 0.0243 - acc: 0.9932 - val_loss: 0.0324 - val_acc: 0.9913
Epoch 3/12
8982/8982 [==============================] - 1s 101us/step - loss: 0.0205 - acc: 0.9942 - val_loss: 0.0331 - val_acc: 0.9917
Epoch 4/12
8982/8982 [==============================] - 1s 108us/step - loss: 0.0186 - acc: 0.9946 - val_loss: 0.0341 - val_acc: 0.9912
Epoch 5/12
8982/8982 [==============================] - 1s 109us/step - loss: 0.0168 - acc: 0.9950 - val_loss: 0.0372 - val_acc: 0.9910
Epoch 6/12
8982/8982 [==============================] - 1s 110us/step - loss: 0.0156 - acc: 0.9954 - val_loss: 0.0358 - val_acc: 0.9913
Epoch 7/12
8982/8982 [==============================] - 1s 109us/step - loss: 0.0140 - acc: 0.9957 - val_loss: 0.0392 - val_acc: 0.9905


# CNN LSTM

## Load & prep data

In [34]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=1000)

X_train = sequence.pad_sequences(X_train, maxlen=100)
X_test = sequence.pad_sequences(X_test, maxlen=100)

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Build model

In [86]:
mdl = Sequential()

mdl.add(Embedding(1000, 128, input_length=100))
mdl.add(Dropout(0.25))

mdl.add(Convolution1D(filters=64, kernel_size=5, padding='valid', activation='relu', strides=1))
mdl.add(MaxPooling1D(pool_size=4))

mdl.add(LSTM(70))
mdl.add(Dense(num_classes))
mdl.add(Activation('sigmoid'))

In [87]:
mdl.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train model

In [88]:
mdl.fit(X_train, y_train, epochs=1, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/1
8982/8982 [==============================] - 32s 4ms/step - loss: 0.1091 - acc: 0.9711 - val_loss: 0.0721 - val_acc: 0.9783


In [89]:
mdl.optimizer.lr = 0.1

In [90]:
mdl.fit(X_train, y_train, epochs=2, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/2
8982/8982 [==============================] - 31s 3ms/step - loss: 0.0671 - acc: 0.9818 - val_loss: 0.0620 - val_acc: 0.9847
Epoch 2/2
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0599 - acc: 0.9846 - val_loss: 0.0593 - val_acc: 0.9848


In [91]:
mdl.optimizer.lr = 0.01

In [92]:
mdl.fit(X_train, y_train, epochs=2, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/2
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0583 - acc: 0.9847 - val_loss: 0.0585 - val_acc: 0.9848
Epoch 2/2
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0574 - acc: 0.9849 - val_loss: 0.0580 - val_acc: 0.9849


In [93]:
mdl.fit(X_train, y_train, epochs=5, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/5
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0566 - acc: 0.9850 - val_loss: 0.0578 - val_acc: 0.9848
Epoch 2/5
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0549 - acc: 0.9857 - val_loss: 0.0562 - val_acc: 0.9860
Epoch 3/5
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0527 - acc: 0.9870 - val_loss: 0.0553 - val_acc: 0.9863
Epoch 4/5
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0509 - acc: 0.9878 - val_loss: 0.0546 - val_acc: 0.9863
Epoch 5/5
8982/8982 [==============================] - 30s 3ms/step - loss: 0.0494 - acc: 0.9883 - val_loss: 0.0537 - val_acc: 0.9867


## Complicating things

### Build model

In [101]:
mdl = Sequential()

mdl.add(Embedding(1000, 128, input_length=100))
mdl.add(Dropout(0.25))

mdl.add(Convolution1D(filters=128, kernel_size=5, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=64, kernel_size=5, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=32, kernel_size=5, padding='valid', activation='relu', strides=1))
mdl.add(MaxPooling1D(pool_size=4))

mdl.add(LSTM(70))
mdl.add(Dense(num_classes))
mdl.add(Activation('sigmoid'))

In [102]:
mdl.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train model

In [103]:
mdl.fit(X_train, y_train, epochs=1, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/1
8982/8982 [==============================] - 43s 5ms/step - loss: 0.1099 - acc: 0.9706 - val_loss: 0.0675 - val_acc: 0.9826


In [105]:
mdl.optimizer.lr = 0.1

In [106]:
mdl.fit(X_train, y_train, epochs=1, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/1
8982/8982 [==============================] - 41s 5ms/step - loss: 0.0616 - acc: 0.9841 - val_loss: 0.0592 - val_acc: 0.9848


In [107]:
mdl.optimizer.lr = 0.01

In [108]:
mdl.fit(X_train, y_train, epochs=5, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/5
8982/8982 [==============================] - 43s 5ms/step - loss: 0.0583 - acc: 0.9846 - val_loss: 0.0581 - val_acc: 0.9848
Epoch 2/5
8982/8982 [==============================] - 43s 5ms/step - loss: 0.0563 - acc: 0.9847 - val_loss: 0.0559 - val_acc: 0.9848
Epoch 3/5
8982/8982 [==============================] - 42s 5ms/step - loss: 0.0528 - acc: 0.9867 - val_loss: 0.0523 - val_acc: 0.9875
Epoch 4/5
8982/8982 [==============================] - 44s 5ms/step - loss: 0.0501 - acc: 0.9878 - val_loss: 0.0524 - val_acc: 0.9870
Epoch 5/5
8982/8982 [==============================] - 44s 5ms/step - loss: 0.0484 - acc: 0.9883 - val_loss: 0.0505 - val_acc: 0.9878


In [109]:
mdl.fit(X_train, y_train, epochs=6, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/6
8982/8982 [==============================] - 44s 5ms/step - loss: 0.0474 - acc: 0.9885 - val_loss: 0.0507 - val_acc: 0.9878
Epoch 2/6
8982/8982 [==============================] - 44s 5ms/step - loss: 0.0465 - acc: 0.9888 - val_loss: 0.0509 - val_acc: 0.9876
Epoch 3/6
8982/8982 [==============================] - 42s 5ms/step - loss: 0.0457 - acc: 0.9889 - val_loss: 0.0503 - val_acc: 0.9878
Epoch 4/6
8982/8982 [==============================] - 40s 4ms/step - loss: 0.0452 - acc: 0.9890 - val_loss: 0.0512 - val_acc: 0.9877
Epoch 5/6
8982/8982 [==============================] - 41s 5ms/step - loss: 0.0443 - acc: 0.9892 - val_loss: 0.0511 - val_acc: 0.9877
Epoch 6/6
8982/8982 [==============================] - 40s 4ms/step - loss: 0.0436 - acc: 0.9892 - val_loss: 0.0502 - val_acc: 0.9877


# More complicating things

trying varying the convolution window size and dropping the LSTM to try to break stagnant training seen in other architectures

In [78]:
mdl = Sequential()

mdl.add(Embedding(1000, 256, input_length=100))

mdl.add(Convolution1D(filters=256, kernel_size=5, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=256, kernel_size=4, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=64, kernel_size=2, padding='valid', activation='relu', strides=1))

mdl.add(MaxPooling1D(pool_size=4))
mdl.add(Dropout(0.5))

mdl.add(Convolution1D(filters=256, kernel_size=5, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=256, kernel_size=4, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1))
mdl.add(Convolution1D(filters=64, kernel_size=2, padding='valid', activation='relu', strides=1))

mdl.add(MaxPooling1D(pool_size=4))
mdl.add(Dropout(0.5))
mdl.add(LSTM(100))

mdl.add(Dense(num_classes))
mdl.add(Activation('sigmoid'))

In [79]:
mdl.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training

In [80]:
mdl.fit(X_train, y_train, epochs=1, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/1
8982/8982 [==============================] - 110s 12ms/step - loss: 0.0923 - acc: 0.9750 - val_loss: 0.0575 - val_acc: 0.9846


In [81]:
mdl.optimizer.lr = 0.1

In [82]:
mdl.fit(X_train, y_train, epochs=4, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/4
8982/8982 [==============================] - 151s 17ms/step - loss: 0.0536 - acc: 0.9864 - val_loss: 0.0533 - val_acc: 0.9867
Epoch 2/4
8982/8982 [==============================] - 147s 16ms/step - loss: 0.0518 - acc: 0.9871 - val_loss: 0.0518 - val_acc: 0.9873
Epoch 3/4
8982/8982 [==============================] - 192s 21ms/step - loss: 0.0506 - acc: 0.9875 - val_loss: 0.0521 - val_acc: 0.9873
Epoch 4/4
8982/8982 [==============================] - 150s 17ms/step - loss: 0.0498 - acc: 0.9877 - val_loss: 0.0525 - val_acc: 0.9872


In [83]:
mdl.optimizer.lr = 0.01

In [84]:
mdl.fit(X_train, y_train, epochs=4, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/4
8982/8982 [==============================] - 179s 20ms/step - loss: 0.0495 - acc: 0.9877 - val_loss: 0.0519 - val_acc: 0.9871
Epoch 2/4
8982/8982 [==============================] - 147s 16ms/step - loss: 0.0487 - acc: 0.9879 - val_loss: 0.0524 - val_acc: 0.9871
Epoch 3/4
8982/8982 [==============================] - 120s 13ms/step - loss: 0.0482 - acc: 0.9879 - val_loss: 0.0519 - val_acc: 0.9872
Epoch 4/4
8982/8982 [==============================] - 130s 14ms/step - loss: 0.0474 - acc: 0.9881 - val_loss: 0.0515 - val_acc: 0.9874


In [85]:
mdl.fit(X_train, y_train, epochs=4, batch_size=28, validation_data=(X_test, y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/4
8982/8982 [==============================] - 111s 12ms/step - loss: 0.0467 - acc: 0.9882 - val_loss: 0.0525 - val_acc: 0.9873
Epoch 2/4
8982/8982 [==============================] - 110s 12ms/step - loss: 0.0457 - acc: 0.9883 - val_loss: 0.0513 - val_acc: 0.9874
Epoch 3/4
8982/8982 [==============================] - 125s 14ms/step - loss: 0.0449 - acc: 0.9884 - val_loss: 0.0514 - val_acc: 0.9873
Epoch 4/4
8982/8982 [==============================] - 134s 15ms/step - loss: 0.0441 - acc: 0.9885 - val_loss: 0.0543 - val_acc: 0.9870
